# Projet : Système de détection de la fatigue oculaire

## Objectif du projet
Ce projet a pour but de développer un modèle de Machine Learning capable de détecter la fatigue oculaire à partir d’images, afin de prévenir les risques liés à la conduite.

---
## Structure du Notebook

Ce notebook est organisé en plusieurs parties principales :

 1- Collecte des données
 
 2- Prétraitement des données
 
 3- Entraînement du modèle
 
 4- Évaluation du modèle
 
 5- Visualisation et interprétation
 
---

## Collecte des données

Cette section contient le code permettant de récupérer et d’organiser les données utilisées dans ce projet.

Le dataset provient de la plateforme **Kaggle** et a été téléchargé manuellement, puis décompressé localement dans le répertoire de travail du projet.

Chaque sous-partie ci-dessous traite une étape spécifique :
- Vérification de la structure des données
- Renommer les sous-dossiers pour améliorer la lisibilité
- Chargement des images depuis les sous-dossiers

In [ ]:
# --- Télécharger le dataset depuis la plateforme Kaggle :
!kaggle datasets download -d ismailnasri20/driver-drowsiness-dataset-ddd
# --- Décompresser localement le dataset dans le répertoire de travail du projet :
import zipfile

with zipfile.ZipFile('driver-drowsiness-dataset-ddd.zip', 'r') as zip_ref:
    zip_ref.extractall('C:/Jupyter_File/Project_Dataset')

In [2]:
# --- Explorer les sous dossiers du dossier de notre dataset :
import os
dataset_path = 'C:/Jupyter_File/Project_dataset/Driver Drowsiness Dataset (DDD)'
files = os.listdir(dataset_path)
print(files)

# --- Renommer le dossier 'Drowsy' en 'yeux_fermés'
os.rename(os.path.join(dataset_path, 'Drowsy'), os.path.join(dataset_path, 'yeux_fermés'))

# --- Renommer le dossier 'Non Drowsy' en 'yeux_ouverts'
os.rename(os.path.join(dataset_path, 'Non Drowsy'), os.path.join(dataset_path, 'yeux_ouverts'))


['yeux_fermés', 'yeux_ouverts']


In [3]:
# --- Chargement des images depuis les sous-dossiers :
import os
from PIL import Image

dataset_path = 'C:/Jupyter_File/Project_dataset/Driver Drowsiness Dataset (DDD)'

# Liste pour stocker les images
images = []
# Liste pour stocker les étiquettes correspondantes
labels = []

# NB: Chaque sous-dossier correspond à une classe :
for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)
    if os.path.isdir(folder_path):  # Vérifier si c'est un dossier
        print(f"Chargement des images de la classe : {folder}")
        
        # Parcourir chaque image dans le sous-dossier
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            try:
                with Image.open(img_path) as img:
                    # Ajouter l'image et son label (nom du sous-dossier) aux listes
                    images.append(img)
                    labels.append(folder)  # L'étiquette est le nom du sous-dossier
            except Exception as e:
                print(f"Erreur avec l'image {img_path}: {e}")

# Afficher le nombre d'images collectées
print(f"Nombre total d'images collectées : {len(images)}")


Chargement des images de la classe : yeux_fermés
Chargement des images de la classe : yeux_ouverts
Nombre total d'images collectées : 41793


## Prétraitement des données

Dans cette section, nous préparons les données brutes pour notre modèle de Machine Learning afin de garantir que ce dernier reçoive des données propres, cohérentes et exploitables.

Les étapes de prétraitement appliquées sont les suivantes :

### 1- Redimensionnement des images
Toutes les images seront redimensionnées à une taille fixe (par exemple 64x64 pixels) afin d’uniformiser les dimensions et réduire le coût de calcul.

### 2- Normalisation
Les valeurs des pixels seront normalisées (généralement entre 0 et 1) pour faciliter l’apprentissage du modèle et améliorer la convergence.

### 3- Attribution des étiquettes
Chaque image sera associée à une étiquette numérique selon sa classe :
- `0` : yeux fermés
- `1` : yeux ouverts

### 4- Division en jeux d’entraînement et de test
Les données seront divisées en deux ensembles :
- **Entraînement** : pour que le modèle apprenne à reconnaître les classes
- **Test** : pour évaluer la performance du modèle sur des données jamais vues
